# La Bretagne et ses stations de vélo autour des voies navigables

Visualisation réalisée avec les fichiers Open-source disponible librement sur le site internet : https://data.bretagne.bzh/

# Import et vérification

In [17]:
import pandas as pd
import numpy as np
import requests
import folium

In [13]:
# Import
stations = pd.read_csv ("stations-de-reparation-et-gonflage-pour-velo-autour-des-voies-navigables-de-bret.csv", sep =';')

In [14]:
stations.head()

,Geo Point,Geo Shape,gml_id,gid,gestionnaire,gonflage,reparation,etat,remarque
0,"48.18152678334775, -1.6407498380429273","{""coordinates"": [-1.640749838042927, 48.181526...",station_reparation_gonflage_velo.1149,1149,Ville de Betton,Oui,Oui,En service,NaN
1,"48.30504360987385, -1.793807825525895","{""coordinates"": [-1.793807825525895, 48.305043...",station_reparation_gonflage_velo.1150,1150,Région Bretagne,Oui,Oui,En service,Installé en mars 2021
2,"48.38394372027351, -1.9800845309505692","{""coordinates"": [-1.980084530950569, 48.383943...",station_reparation_gonflage_velo.1151,1151,Région Bretagne,Oui,Oui,En service,Installé en mars 2021
3,"47.926273836345054, -1.7489683151492137","{""coordinates"": [-1.748968315149214, 47.926273...",station_reparation_gonflage_velo.1152,1152,Région Bretagne,Oui,Oui,En service,Installé en mars 2021
4,"47.67903053282296, -1.9768359315837265","{""coordinates"": [-1.976835931583727, 47.679030...",station_reparation_gonflage_velo.1153,1153,Région Bretagne,Oui,Oui,En service,Installé en mars 2021


# Traitement

In [15]:
# Traitement des coordonées géographiques
stations[['Latitude', 'Longitude']] = stations['Geo Point'].str.split(',', expand=True)
stations[['Latitude', 'Longitude']] = stations[['Latitude', 'Longitude']].astype(float)

# Suppression des Coordonnées initiales
stations.drop('Geo Point', axis=1, inplace=True)

# Visualisation géographique

In [19]:
# Fonction pour récupérer le nom de la commune à partir des coordonnées GPS
def get_commune(lat, lon):
    url = f"https://nominatim.openstreetmap.org/reverse?format=jsonv2&lat={lat}&lon={lon}"
    response = requests.get(url).json()
    if 'city' in response['address']:
        return response['address']['city']
    elif 'town' in response['address']:
        return response['address']['town']
    elif 'village' in response['address']:
        return response['address']['village']
    elif 'hamlet' in response['address']:
        return response['address']['hamlet']
    else:
        return "Unknown"
# create a map centered on the mean latitude and longitude
mean_lat = stations['Latitude'].mean()
mean_lon = stations['Longitude'].mean()
m = folium.Map(location=[mean_lat, mean_lon], zoom_start=10)

# add a marker layer to the map
for idx, row in stations.iterrows():
    # get the latitude, longitude, and circle size for the current row
    lat = row['Latitude']
    lon = row['Longitude']
    
    # add a marker to the map with a popup displaying the exact coordinates and the name of the commune
    commune = get_commune(lat, lon)
    popup_content = f"Lat: {lat}, Lon: {lon}<br>Commune: {commune}"
    folium.Marker(location=[lat, lon], popup=popup_content).add_to(m)

# display the map
m